Forgotten ones:


In [ ]:
function despawn!(agent, model)
    # Despawn agent if unattractive steps exceed threshold
    remove_agent!(agent, model)
end

# Agent-Based Video Distortion (ABVD)

![Structure of the Digital Petri Dish](../assets/notebook-images/Structure.jpg)

## Introduction

Our project is on a special method we developed for video distortion. Most distortion tools translate the color values of the source pixel or exploit the quirks in compression algorithms to create what's called 'data moshing'. However, independently the term data moshing evokes crowds of people pushing into one another at a punk show—got us thinking if we could use agent-based modeling to make pixels mosh.

To do this, we thought about an image as a petri dish, filled with all different kinds of nutrients and poisons, and the pixels as bacteria moving towards nutrients and away from poisons. Then, video frames are just a snapshot of a petri dish in time, followed by another petri dish in time.

The goal, then, is to create an agent (or bacteria) that will create visually interesting effects.

We've created a system where pixels behave like microscopic organisms, responding to their environment in ways that create emergent visual patterns. Each video frame becomes a moment in time of this digital ecosystem.

## The Digital Petri Dish

In our model, each frame is treated as a nutrient environment where:

- Light areas are "nutrients" that attract our pixel-organisms
- Dark areas are "toxic" zones that organisms try to avoid
- Pixels can mutate their colors based on environmental pressures
- Population dynamics emerge from birth and death cycles

This biological metaphor creates organic, flowing distortions that feel more alive than traditional effects.

## Loading in Necessary Packages

We start by loading in all of our necessary packages.

In [ ]:
using Pkg

required_packages = [
    "CSV",
    "DataFrames",
    "VideoIO",
    "Images",
    "Logging",
    "Agents",
    "ProgressMeter",
    "FileIO"
]

for package in required_packages
    try
        @eval using $(Symbol(package))
    catch
        println("Installing $package...")
        Pkg.add(package)
        @eval using $(Symbol(package))
    end
end

println("All required packages are installed and loaded successfully!")

using CSV
using DataFrames
using VideoIO
using Images
using Logging
using Agents
using ProgressMeter
using FileIO

### Explanation

This code block ensures that all necessary packages are installed and loaded. It iterates over a list of required packages, attempting to load each one. If a package is not installed, it is automatically added and then loaded. This guarantees that the environment is correctly set up for the subsequent steps. The output will confirm the successful installation and loading of all packages.

## The Pixel Organism

Just as bacteria have specific traits that help them survive in their environment, our pixel agents have key properties:

1. **HSL Color Value** (hsl_value)
   - Like bacterial pigmentation
   - Adapts to environmental conditions
   - Can mutate over time

2. **Active State** (active)
   - Represents organism viability
   - Similar to bacterial life/death states
   - Determines if agent survives to next generation

3. **Unattractive Steps** (unattractive_steps)
   - Tracks exposure to "toxic" (dark) areas
   - Similar to bacterial stress response
   - Influences survival probability

These properties work together to create organisms that can adapt and evolve within our digital petri dish, much like real bacteria adapting to their environment.

In [ ]:
@agent PixelAgent GridAgent{2} begin
    hsl_value::HSL
    active::Bool
    unattractive_steps::Int
end

dump(PixelAgent)

### Explanation

The `PixelAgent` struct is defined with three properties: `hsl_value` for color representation, `active` to indicate if the agent is currently active, and `unattractive_steps` to track exposure to undesirable areas. This structure forms the basis for agent behavior in the model. The output will show the successful definition of the `PixelAgent` struct.

## Utility Functions

In this section, we introduce utility functions that are crucial for the movement and decision-making processes of our pixel agents. These functions enable the agents to navigate their environment effectively, responding to the lightness of their surroundings and making decisions that influence their movement patterns.

### Inactive Test Function

The `inactive_test` function adds an element of randomness to the despawning of agents. It evaluates whether an agent should be marked inactive based on its exposure to "toxic" areas, represented by dark pixels. This function uses a probabilistic approach, influenced by the agent's unattractive steps and a global parameter, to determine if an agent should be removed from the simulation.

### Motivated Move Function

The `motivated_move` function is designed to simulate a gradient ascent behavior, where pixel agents are attracted to brighter areas of the frame. This function evaluates potential positions and selects the one with the highest lightness value, encouraging agents to move towards "nutrient-rich" areas.

### Move Function

The `move!` function governs the movement logic of each pixel agent. It incorporates randomness to prevent stagnation and uses the lightness of the current pixel to determine movement strategy. The function also checks if an agent should be marked inactive using the `inactive_test` function.

![Movement Model](../assets/notebook-images/move.jpg)

The graphic above illustrates the movement model. Agents are influenced by the lightness of their environment:

- **Unmotivated Movement (M1):** Occurs in the lightest areas, where agents make small, random movements.
- **Motivated Movement (M2):** In light areas, agents are more directed, seeking brighter pixels.
- **Motivated Movement (M3):** In darker areas, agents make longer, directed movements to find brighter regions.

This visual representation helps to conceptualize how agents navigate through different light zones, adapting their behavior based on environmental conditions.

In [ ]:
function motivated_move(current_pos, positions, hsl_matrix)
    max_lightness = 0
    max_position = rand(positions)

    for (y, x) in positions
        lightness = hsl_matrix[y, x].l
        if lightness > max_lightness
            max_lightness = lightness
            max_position = (y, x)
        end
    end

    return max_position
end

function move!(agent, model, hsl_matrix)
    pixel_color = hsl_matrix[agent.pos[1], agent.pos[2]]

    if inactive_test(agent.unattractive_steps, pixel_color)
        agent.active = false
    else
        neighboring_positions = collect(nearby_positions(agent, model, rand(1:3)))

        if rand() < 0.10
            move_agent!(agent, rand(neighboring_positions), model)
        elseif pixel_color.l > 0.7
            neighboring_positions = collect(nearby_positions(agent, model, 1))
            move_agent!(agent, rand(neighboring_positions), model)
        elseif pixel_color.l > 0.5
            neighboring_positions = collect(nearby_positions(agent, model, 1))
            if !isempty(neighboring_positions)
                new_pos = motivated_move(agent.pos, neighboring_positions, hsl_matrix)
                move_agent!(agent, new_pos, model)
            end
        else
            neighboring_positions = collect(nearby_positions(agent, model, 3))
            if !isempty(neighboring_positions)
                new_pos = motivated_move(agent.pos, neighboring_positions, hsl_matrix)
                move_agent!(agent, new_pos, model)
            end
        end
    end
end

function inactive_test(steps, hsl)
    global parameters
    return rand() < (steps * parameters[:unattractive_tolerance])
end

### Detailed Explanation

The `motivated_move` function iterates over all possible positions an agent can move to, evaluating the lightness of each position. It selects the position with the highest lightness, simulating a natural attraction to brighter areas.

The `move!` function is more complex, incorporating several movement strategies based on the lightness of the current pixel:

- **Random Movement:** Introduces a 10% chance for the agent to move randomly, which helps in exploring new areas and prevents the system from becoming static.
- **Brightness-Based Movement:** Adjusts the movement strategy based on the lightness of the current pixel:
  - **Very Bright Areas (L > 0.7):** Agents make small random movements, simulating a stable environment.
  - **Moderately Bright Areas (L > 0.5):** Agents perform medium-range directed movements towards brighter pixels.
  - **Dark Areas (L ≤ 0.5):** Agents engage in long-range directed movements, searching for brighter areas.

The `inactive_test` function plays a crucial role in this system by determining when an agent should be marked inactive, adding a layer of realism to the simulation by mimicking the natural lifecycle of organisms. This movement logic, combined with the motivated move function, creates a dynamic and adaptive system where pixel agents continuously seek out optimal conditions, contributing to the emergent visual patterns in the video distortion process.

## Color Mutation System

The `mutate_color!` function is a key component of the color mutation system, simulating how pixel agents adapt to their environment. This function uses the HSL (Hue, Saturation, Lightness) color space to allow for more natural-looking color transitions compared to RGB. The mutation system is designed to reflect different environmental pressures and includes several distinct behaviors:

1. **Random Color Jumps:** A small chance of complete color randomization, simulating sudden mutations or "quantum jumps."
2. **Environmental Adaptation:** In brighter areas, colors gradually adapt to their surroundings, with hue shifting towards local pixel colors.
3. **Dark Area Behavior:** In darker areas, colors become more unstable, with higher mutation rates and a tendency towards darker values.

This function is crucial for creating the dynamic and organic visual effects seen in the video distortion process.

In [ ]:
function mutate_color!(agent, p_hsl)
    global parameters
    mutation_strength = parameters[:mutation_strength]
    mutation_rate = parameters[:mutation_rate]
    a_hsl = agent.hsl_value

    old_hue, old_saturation, old_lightness = a_hsl.h, a_hsl.s, a_hsl.l

    if rand() < parameters[:random_color_rate]
        agent.hsl_value = HSL(rand() * 360, rand(), rand() * 0.5)
    elseif a_hsl.l > 0.3
        agent.hsl_value = HSL(
            clamp(a_hsl.h + (p_hsl.h - a_hsl.h) * mutation_rate, 0, 360),
            clamp((a_hsl.s + p_hsl.s) / 2, 0.5, 1),
            clamp((a_hsl.l + p_hsl.l) / 2, 0, 1)
        )
    else
        agent.hsl_value = HSL(
            clamp(a_hsl.h + rand() * 360 * mutation_strength, 0, 360),
            clamp(a_hsl.s + (rand() - 0.5) * mutation_strength, 0.5, 1),
            clamp(a_hsl.l + (rand() - 0.5) * mutation_strength - 0.01, 0.1, 0.5)
        )
    end
end

### Explanation

The `mutate_color!` function adjusts the HSL values of an agent based on its environment. It introduces randomness to simulate sudden mutations and adapts colors gradually in brighter areas. In darker areas, the mutation strength increases, causing more significant changes. The output will confirm the successful definition of the color mutation function.

## Initialization of the ABM Model

The ABM model is initialized with the first frame of the video, setting up the environment and agents.

In [ ]:
function initialize_abm_model(input_image, population_mod)
    hsl_matrix = convert(Matrix{HSL}, input_image)
    frame_dims = size(hsl_matrix)
    grid_dims = (frame_dims[1], frame_dims[2])

    model = ABM(PixelAgent, GridSpace(grid_dims, periodic=false))
    global agent_id_counter = 1
    position_to_agent = Dict()

    for y in 1:population_mod:frame_dims[1], x in 1:population_mod:frame_dims[2]
        hsl_value = hsl_matrix[y, x]
        agent = PixelAgent(agent_id_counter, (y, x), hsl_value, true, 0)
        add_agent!(agent, model)
        position_to_agent[(y, x)] = agent
        global agent_id_counter += 1
    end

    log_message("Model initialized with $(nagents(model)) agents.")
    return model, grid_dims, agent_id_counter, position_to_agent
end

### Explanation

The `initialize_abm_model` function converts the input image into an HSL matrix and sets up a grid space for the agents. Agents are created at specified intervals, and their initial properties are defined. This setup is crucial for simulating the behavior of pixel agents. The output will confirm the successful initialization of the ABM model with a log message indicating the number of agents created.

## Apply ABM Distortion

The `apply_abm_distortion` function processes an input image by simulating the behavior of agents on a grid. Each agent represents a pixel that can move and change color based on its environment. The function updates the positions and colors of the agents, creating a distorted version of the input image. This distortion is achieved by treating the image as a dynamic environment where agents interact with their surroundings, leading to emergent visual patterns.

In [ ]:
function apply_abm_distortion(input_image, model, agent_id_counter)
    hsl_matrix = convert(Matrix{HSL}, input_image)
    frame_dims = size(hsl_matrix)

    despawn_counter = 0
    global parameters

    # Initialize the distorted frame with a default color
    distorted_frame = fill(RGB(0, 0, 0), frame_dims[1], frame_dims[2])

    for agent in allagents(model)
        old_pos = agent.pos  # Store the old position

        move!(agent, model, hsl_matrix)
        mutate_color!(agent, hsl_matrix[agent.pos[1], agent.pos[2]])

        if !agent.active
            despawn!(agent, model)
            despawn_counter += 1
        end

        if hsl_matrix[agent.pos[1], agent.pos[2]].l < parameters[:unattractive_threshold]
            agent.unattractive_steps += 1
        else
            agent.unattractive_steps = 0
        end

        # Update the current position with the agent's color
        color = RGB(agent.hsl_value)
        for i in 0:parameters[:pixel_size]-1
            for j in 0:parameters[:pixel_size]-1
                y = clamp(agent.pos[1] + i, 1, frame_dims[1])
                x = clamp(agent.pos[2] + j, 1, frame_dims[2])
                distorted_frame[y, x] = color
            end
        end

        # Clear the old position if trailing is false
        if !parameters[:trailing] && old_pos != agent.pos
            for i in 0:parameters[:pixel_size]-1
                for j in 0:parameters[:pixel_size]-1
                    y = clamp(old_pos[1] + i, 1, frame_dims[1])
                    x = clamp(old_pos[2] + j, 1, frame_dims[2])
                    distorted_frame[y, x] = RGB(0, 0, 0)  # Clear the trail
                end
            end
        end
    end

    return colorview(RGB, distorted_frame)
end

### Explanation

The `apply_abm_distortion` function operates as follows:

1. **HSL Conversion**: The input image is converted into an HSL matrix, which allows for more natural manipulation of color properties.

2. **Frame Initialization**: A new frame is initialized with a default black color, which will be updated as agents move and change color.

3. **Agent Processing**: Each agent in the model is processed:
   - **Movement**: The agent's position is updated based on its environment using the `move!` function.
   - **Color Mutation**: The agent's color is potentially mutated based on the lightness of its current position using the `mutate_color!` function.
   - **Despawn Check**: If an agent becomes inactive, it is removed from the model.
   - **Unattractive Steps**: The agent's exposure to "toxic" areas (dark pixels) is tracked, influencing its survival.

4. **Color Update**: The agent's new position is updated in the distorted frame with its current color. If the `trailing` parameter is false, the agent's previous position is cleared to prevent trails.

5. **Return**: The function returns a color view of the distorted frame, which represents the processed image with applied distortions.

This function is central to creating the dynamic and organic visual effects characteristic of the ABVD model.

## Application of ABM Distortion to Video

The ABM distortion is applied to each video frame, processing the frames through the simulation.

In [ ]:
function apply_abm_distortion_to_video(input_video_path; parameters)
    frame_folder = "../outputs/frames_temp/"
    processed_folder = "../outputs/processed_frames_temp/"

    isdir(frame_folder) || mkpath(frame_folder)
    isdir(processed_folder) || mkpath(processed_folder)

    log_message("Processing video $input_video_path")

    video = VideoIO.openvideo(input_video_path)
    frame_index = 1
    while !eof(video)
        frame = read(video)
        save("$frame_folder/frame_$frame_index.png", frame)
        frame_index += 1
    end
    close(video)
    log_message("Extracted $(frame_index - 1) frames from $input_video_path.")

    first_frame = load("$frame_folder/frame_1.png")
    global model, grid_dims, agent_id_counter, position_to_agent = initialize_abm_model(first_frame, parameters[:pixel_pop_mod])

    # Create a progress bar
    progress = Progress(frame_index - 1, 1, "Processing frames")

    for i in 1:frame_index - 1
        original_frame = load("$frame_folder/frame_$i.png")
        distorted_frame = apply_abm_distortion(original_frame, model, agent_id_counter)
        save("$processed_folder/frame_$i.png", distorted_frame)
        
        # Update progress bar
        next!(progress)
        
        # Log every 10 frames
        if i % 10 == 0
            flush(stdout)  # Ensure the output is immediately displayed
        end
    end

    # Log a summary of the processing
    log_message("Completed processing of $frame_index frames.")
end

### Explanation

The `apply_abm_distortion_to_video` function extracts frames from the input video, applies the ABM distortion to each frame, and saves the processed frames. It initializes the model with the first frame and processes each subsequent frame through the simulation, updating agent positions and colors. The output will include log messages indicating the progress of frame extraction and processing.

## Compilation of Video from Frames

The processed frames are compiled back into a video, completing the distortion process.

In [ ]:
function compile_video_from_frames(input_folder, output_video_path, fps=30)
    println("Starting video compilation...")
    python_path = "/Users/alexsciuto/Library/Mobile Documents/com~apple~CloudDocs/DataWithAlex/ABM-Project-Frank-Alex-COP6526/ABVD-Frank-Alex-COP6526/venv/bin/python"
    script_path = "../video-compiler.py"  # Adjusted path to the script
    try
        # Use the correct path for the processed frames
        command = `$python_path $script_path ../outputs/processed_frames_temp $output_video_path --fps $fps`
        
        println("Running Python script to compile video...")
        println("Command: $command")

        output = read(command, String)
        println("Python script output:\n$output")
        log_message("Python script output:\n$output")

        log_message("Video saved to $output_video_path.")
        println("Video saved to $output_video_path.")
    catch e
        log_message("Error executing Python script: $e")
        println("Error executing Python script: $e")
    end
end

### Explanation

The `compile_video_from_frames` function uses a Python script to compile the processed frames into a final video. It logs the process and handles any errors that occur. This step completes the video distortion pipeline, resulting in a visually compelling video that showcases the effects of the ABM model. The output will include log messages confirming the successful compilation of the video.

## Running Example

In this section, we will demonstrate the application of the ABVD model using a sample video. We will set parameters optimized for speed and detail, process the video frames, and compile the final output video.

### Parameters and Paths

We define the parameters and paths necessary for processing the video. These parameters control the behavior of the pixel agents and the characteristics of the output video.

In [ ]:
# Parameters optimized for speed
global parameters = Dict(
    :pixel_size => 8,          # Smaller pixel size for more detail
    :pixel_pop_mod => 10,      # More initial agents
    :sim_per_frames => 1,      # Keep as is
    :mutation_strength => 0.6, # Increase for more color variation
    :unattractive_tolerance => 0.02,  # Lower tolerance to reduce despawning
    :unattractive_threshold => 0.2,   # Lower threshold to keep agents active longer
    :mutation_rate => 0.5,     # Increase for more color change
    :random_color_rate => 0.1, # Increase for more random color generation
    :trailing => false         # Keep as is
)

# Define paths relative to the notebook's location
input_video_path = "../inputs/eye.mp4"

# Extract the base name of the input video file
input_base_name = splitext(basename(input_video_path))[1]

# Create a string representation of all parameters
param_str = "px$(parameters[:pixel_size])_pop$(parameters[:pixel_pop_mod])_sim$(parameters[:sim_per_frames])_" *
            "mutStr$(parameters[:mutation_strength])_unTol$(parameters[:unattractive_tolerance])_" *
            "unThr$(parameters[:unattractive_threshold])_mutRate$(parameters[:mutation_rate])_" *
            "randColRate$(parameters[:random_color_rate])"

# Construct the output video path
output_video_path = "../outputs/$(input_base_name)_$(param_str).mp4"

### Processing and Compiling the Video

We apply the ABVD model to the video frames and compile the processed frames into a final video. This process involves extracting frames, applying agent-based distortion, and using a Python script to compile the frames into a video.

In [ ]:
# Define logging function with log levels
function log_message(msg, level="INFO")
    if level == "ERROR" || level == "WARNING" || level == "INFO"
        println("[$level] $msg")
    end
end

# Process frames
apply_abm_distortion_to_video(input_video_path; parameters)

# Use the correct path for the processed frames
compile_video_from_frames("../outputs/processed_frames_temp", output_video_path, 30)

### Expected Output

The following output is expected when running the example. It includes log messages indicating the progress of video processing and compilation, as well as the final confirmation of the video being saved.


## Conclusion

This notebook demonstrates the implementation of the Agent-Based Video Distortion (ABVD) model, showcasing how agent-based modeling can be applied to video processing to create dynamic and organic visual effects. By treating each video frame as a digital petri dish, we simulate pixel agents that respond to their environment, resulting in emergent visual patterns that are both complex and aesthetically intriguing.

The detailed markdown explanations throughout the notebook provide a comprehensive understanding of each step, from setting up the environment and defining pixel agents to processing video frames and compiling the final output. This ensures that the notebook is not only executable but also serves as an educational resource for understanding the principles and applications of agent-based modeling in digital media.

By experimenting with different parameters and input videos, users can explore a wide range of visual effects, further demonstrating the versatility and creative potential of the ABVD model.

In [ ]:
# Parameters optimized for speed
global parameters = Dict(
    :pixel_size => 8,          # Smaller pixel size for more detail
    :pixel_pop_mod => 10,      # More initial agents
    :sim_per_frames => 1,      # Keep as is
    :mutation_strength => 0.6, # Increase for more color variation
    :unattractive_tolerance => 0.02,  # Lower tolerance to reduce despawning
    :unattractive_threshold => 0.2,   # Lower threshold to keep agents active longer
    :mutation_rate => 0.5,     # Increase for more color change
    :random_color_rate => 0.1, # Increase for more random color generation
    :trailing => true          # Keep as is
)

# Define paths relative to the notebook's location
input_video_path = "../inputs/eye.mp4"

# Extract the base name of the input video file
input_base_name = splitext(basename(input_video_path))[1]

# Create a string representation of all parameters
param_str = "px$(parameters[:pixel_size])_pop$(parameters[:pixel_pop_mod])_sim$(parameters[:sim_per_frames])_" *
            "mutStr$(parameters[:mutation_strength])_unTol$(parameters[:unattractive_tolerance])_" *
            "unThr$(parameters[:unattractive_threshold])_mutRate$(parameters[:mutation_rate])_" *
            "randColRate$(parameters[:random_color_rate])_trailing$(parameters[:trailing])"

# Construct the output video path
output_video_path = "../outputs/$(input_base_name)_$(param_str).mp4"

# Process frames
apply_abm_distortion_to_video(input_video_path; parameters)

# Use the correct path for the processed frames
compile_video_from_frames("../outputs/processed_frames_temp", output_video_path, 30)